# Transformers and Magnetic Components

This notebook covers transformer and inductor modeling in Pulsim for power electronics.

## Contents
1. Ideal Transformer Model
2. Real Transformer Parameters
3. Magnetizing Inductance Effects
4. Leakage Inductance
5. Flyback Transformer
6. Coupled Inductors
7. Core Saturation (Advanced)

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

## 1. Ideal Transformer Model

An ideal transformer has:
- Perfect coupling (k = 1)
- No losses
- Infinite magnetizing inductance

$$\frac{V_1}{V_2} = \frac{N_1}{N_2} = n$$

$$\frac{I_2}{I_1} = n$$

$$V_1 \cdot I_1 = V_2 \cdot I_2 \quad \text{(Power conservation)}$$

```python
TransformerParams:
    turns_ratio = 1.0   # N1:N2 (primary to secondary)
    lm = 0.0            # Magnetizing inductance (0 = ideal)
    ll1 = 0.0           # Primary leakage inductance
    ll2 = 0.0           # Secondary leakage inductance
```

In [ ]:
# Create ideal transformer
xfmr = pulsim.TransformerParams()
xfmr.turns_ratio = 2.0  # 2:1 step-down
xfmr.lm = 0.0           # Infinite magnetizing inductance (ideal)
xfmr.ll1 = 0.0          # No leakage
xfmr.ll2 = 0.0

print("Ideal Transformer:")
print(f"  Turns ratio (N1:N2): {xfmr.turns_ratio}:1")
print(f"  Voltage ratio: V1/V2 = {xfmr.turns_ratio}")
print(f"  Current ratio: I2/I1 = {xfmr.turns_ratio}")

In [ ]:
# Simulate ideal transformer with AC input
netlist = '''
{
  "name": "Ideal Transformer",
  "components": [
    {"type": "V", "name": "Vpri", "nodes": ["p1", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 120, "frequency": 60}},
    {"type": "T", "name": "T1", "nodes": ["p1", "0", "s1", "0"],
     "params": {"turns_ratio": 2.0, "lm": 0}},
    {"type": "R", "name": "Rload", "nodes": ["s1", "0"], "value": 10}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 50e-3  # 3 cycles at 60Hz
options.dt = 10e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3
v_pri = np.array(data['signals']['V(p1)'])
v_sec = np.array(data['signals']['V(s1)'])

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(time_ms, v_pri, 'b-', linewidth=1.5, label='Primary (120V peak)')
axes[0].plot(time_ms, v_sec, 'r-', linewidth=1.5, label='Secondary (60V peak)')
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Ideal Transformer: 2:1 Step-Down')
axes[0].legend()
axes[0].grid(True)

# Calculate currents
i_sec = v_sec / 10  # Rload = 10Ω
i_pri = i_sec / 2   # Current steps up by turns ratio

axes[1].plot(time_ms, i_pri, 'b-', linewidth=1.5, label='Primary current')
axes[1].plot(time_ms, i_sec, 'r-', linewidth=1.5, label='Secondary current')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Current (A)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Verify voltage ratio
ratio = np.max(v_pri) / np.max(v_sec)
print(f"\nMeasured voltage ratio: {ratio:.2f}:1")
print(f"Expected: 2.00:1")

## 2. Real Transformer Parameters

Real transformers have non-ideal characteristics:

```
     Ll1        Ideal         Ll2
  ┌──LLLL──┬──┤ n:1 ├──┬──LLLL──┐
  │        │           │        │
P1│        Lm          │        │S1
  │        │           │        │
  └────────┴───────────┴────────┘
         P2                    S2
```

- **Lm**: Magnetizing inductance (finite, stores energy)
- **Ll1**: Primary leakage inductance (uncoupled flux)
- **Ll2**: Secondary leakage inductance

In [ ]:
# Create realistic transformer
xfmr_real = pulsim.TransformerParams()
xfmr_real.turns_ratio = 2.0   # 2:1 step-down
xfmr_real.lm = 10e-3          # 10mH magnetizing inductance
xfmr_real.ll1 = 100e-6        # 100µH primary leakage
xfmr_real.ll2 = 25e-6         # 25µH secondary leakage (reflects as 100µH to primary)

print("Real Transformer Parameters:")
print(f"  Turns ratio: {xfmr_real.turns_ratio}:1")
print(f"  Magnetizing inductance (Lm): {xfmr_real.lm*1e3:.1f} mH")
print(f"  Primary leakage (Ll1): {xfmr_real.ll1*1e6:.0f} µH")
print(f"  Secondary leakage (Ll2): {xfmr_real.ll2*1e6:.0f} µH")

# Calculate coupling coefficient
# k = Lm / sqrt((Lm + Ll1) * (Lm/n² + Ll2))
Lm = xfmr_real.lm
Ll1 = xfmr_real.ll1
Ll2 = xfmr_real.ll2
n = xfmr_real.turns_ratio

L1 = Lm + Ll1  # Total primary inductance
L2 = Lm/n**2 + Ll2  # Total secondary inductance (referred)
k = Lm / np.sqrt(L1 * (Lm + n**2 * Ll2))

print(f"\nDerived parameters:")
print(f"  Total primary inductance: {L1*1e3:.2f} mH")
print(f"  Total secondary inductance: {L2*1e3:.3f} mH")
print(f"  Coupling coefficient k: {k:.4f}")

## 3. Magnetizing Inductance Effects

The magnetizing inductance draws current even with no load:

$$I_m = \frac{V_1}{j\omega L_m}$$

This creates:
- Magnetizing current (reactive)
- Core losses (if resistance modeled)
- Limits low-frequency operation

In [ ]:
# Compare ideal vs real transformer (magnetizing current)
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Ideal (very high Lm)
netlist_ideal = '''
{
  "name": "Ideal Transformer",
  "components": [
    {"type": "V", "name": "Vpri", "nodes": ["p1", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 120, "frequency": 60}},
    {"type": "T", "name": "T1", "nodes": ["p1", "0", "s1", "0"],
     "params": {"turns_ratio": 2.0, "lm": 1.0}}
  ]
}
'''

# Real (finite Lm)
netlist_real = '''
{
  "name": "Real Transformer",
  "components": [
    {"type": "V", "name": "Vpri", "nodes": ["p1", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 120, "frequency": 60}},
    {"type": "T", "name": "T1", "nodes": ["p1", "0", "s1", "0"],
     "params": {"turns_ratio": 2.0, "lm": 0.1}}
  ]
}
'''

options = pulsim.SimulationOptions()
options.tstop = 50e-3
options.dt = 10e-6

for idx, (netlist, title) in enumerate([(netlist_ideal, 'Ideal (Lm = 1H)'), 
                                         (netlist_real, 'Real (Lm = 100mH)')]):
    circuit = pulsim.parse_netlist_string(netlist)
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    time_ms = np.array(data['time']) * 1e3
    v_pri = np.array(data['signals']['V(p1)'])
    
    # Primary current (magnetizing only, no load)
    i_pri = np.array(data['signals'].get('I(Vpri)', np.zeros_like(v_pri)))
    
    axes[0, idx].plot(time_ms, v_pri, 'b-', linewidth=1.5)
    axes[0, idx].set_ylabel('Voltage (V)')
    axes[0, idx].set_title(f'{title} - Primary Voltage')
    axes[0, idx].grid(True)
    
    axes[1, idx].plot(time_ms, -i_pri, 'r-', linewidth=1.5)
    axes[1, idx].set_xlabel('Time (ms)')
    axes[1, idx].set_ylabel('Magnetizing Current (A)')
    axes[1, idx].set_title(f'{title} - Magnetizing Current (no load)')
    axes[1, idx].grid(True)

plt.tight_layout()
plt.show()

# Calculate expected magnetizing current
Vpk = 120
f = 60
Lm_ideal = 1.0
Lm_real = 0.1

Im_ideal = Vpk / (2 * np.pi * f * Lm_ideal)
Im_real = Vpk / (2 * np.pi * f * Lm_real)

print(f"Expected magnetizing current (peak):")
print(f"  Ideal (Lm=1H): {Im_ideal:.3f} A")
print(f"  Real (Lm=100mH): {Im_real:.2f} A")

## 4. Leakage Inductance

Leakage inductance causes:
- Voltage spikes during switching
- Energy storage that must be managed
- Resonance with parasitic capacitance

In power converters, leakage energy can be:
- Dissipated in snubbers
- Recovered (active clamp)
- Used for soft-switching (resonant converters)

In [ ]:
# Demonstrate leakage inductance effect during switching
netlist = '''
{
  "name": "Transformer with Leakage",
  "components": [
    {"type": "V", "name": "Vdc", "nodes": ["vdc", "0"], "value": 48},
    {"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": 100e3, "duty": 0.4}},
    {"type": "S", "name": "S1", "nodes": ["vdc", "p1", "pwm", "0"],
     "params": {"ron": 0.01, "roff": 1e9, "vth": 5}},
    {"type": "T", "name": "T1", "nodes": ["p1", "0", "s1", "0"],
     "params": {"turns_ratio": 4.0, "lm": 1e-3, "ll1": 10e-6, "ll2": 0.625e-6}},
    {"type": "D", "name": "D1", "nodes": ["s1", "out"],
     "params": {"ideal": false, "is": 1e-10, "n": 1.5}},
    {"type": "C", "name": "Cout", "nodes": ["out", "0"], "value": 100e-6},
    {"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": 5}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 50e-6  # 5 switching cycles
options.dt = 5e-9      # Fine resolution for spikes

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

# Switch node voltage
axes[0].plot(time_us, data['signals']['V(p1)'], 'b-', linewidth=1)
axes[0].set_ylabel('V(p1) (V)')
axes[0].set_title('Primary Switch Node - Leakage Inductance Voltage Spikes')
axes[0].grid(True)

# Secondary voltage
axes[1].plot(time_us, data['signals']['V(s1)'], 'r-', linewidth=1)
axes[1].set_ylabel('V(s1) (V)')
axes[1].set_title('Secondary Voltage')
axes[1].grid(True)

# Output voltage
axes[2].plot(time_us, data['signals']['V(out)'], 'g-', linewidth=1.5)
axes[2].set_xlabel('Time (µs)')
axes[2].set_ylabel('V(out) (V)')
axes[2].set_title('Output Voltage')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("Leakage inductance causes voltage spikes at switch turn-off")
print("Energy stored in leakage: E = ½ Ll × I²")

## 5. Flyback Transformer

A flyback transformer is actually a coupled inductor:
- Energy stored in air gap during on-time
- Energy transferred to secondary during off-time
- Primary and secondary don't conduct simultaneously

$$V_{out} = V_{in} \cdot \frac{D}{1-D} \cdot \frac{N_s}{N_p}$$

In [ ]:
# Flyback converter simulation
Vin = 48
Vout_target = 12
D = 0.3  # Duty cycle
n = 4    # Turns ratio Np:Ns
fsw = 100e3

# Expected output
Vout_calc = Vin * D / (1 - D) / n
print(f"Flyback Converter Design:")
print(f"  Input: {Vin} V")
print(f"  Turns ratio: {n}:1")
print(f"  Duty cycle: {D*100:.0f}%")
print(f"  Expected output: {Vout_calc:.1f} V")

In [ ]:
# Flyback transformer simulation
netlist = f'''
{{
  "name": "Flyback Converter",
  "components": [
    {{"type": "V", "name": "Vin", "nodes": ["vin", "0"], "value": {Vin}}},
    {{"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
     "waveform": {{"type": "pwm", "v_off": 0, "v_on": 10, 
                  "frequency": {fsw}, "duty": {D}}}}},
    
    {{"type": "S", "name": "S1", "nodes": ["drain", "0", "pwm", "0"],
     "params": {{"ron": 0.05, "roff": 1e9, "vth": 5}}}},
    
    {{"type": "T", "name": "T1", "nodes": ["vin", "drain", "0", "s_anode"],
     "params": {{"turns_ratio": {n}, "lm": 500e-6, "ll1": 5e-6, "ll2": 0.3e-6}}}},
    
    {{"type": "D", "name": "D1", "nodes": ["s_anode", "vout"],
     "params": {{"ideal": false, "is": 1e-10, "n": 1.3}}}},
    
    {{"type": "C", "name": "Cout", "nodes": ["vout", "0"], "value": 220e-6}},
    {{"type": "R", "name": "Rload", "nodes": ["vout", "0"], "value": 6}}
  ]
}}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 500e-6  # 50 cycles
options.dt = 20e-9

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

# Switch drain voltage
axes[0].plot(time_us, data['signals']['V(drain)'], 'b-', linewidth=0.8)
axes[0].set_ylabel('V(drain) (V)')
axes[0].set_title('Flyback Converter Waveforms')
axes[0].grid(True)

# Secondary voltage
axes[1].plot(time_us, data['signals']['V(s_anode)'], 'r-', linewidth=0.8)
axes[1].set_ylabel('V(secondary) (V)')
axes[1].grid(True)

# Output voltage
v_out = np.array(data['signals']['V(vout)'])
axes[2].plot(time_us, v_out, 'g-', linewidth=1.5)
axes[2].axhline(y=Vout_calc, color='orange', linestyle='--', label=f'Expected: {Vout_calc:.1f}V')
axes[2].set_xlabel('Time (µs)')
axes[2].set_ylabel('V(out) (V)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Steady-state output
v_out_ss = np.mean(v_out[-1000:])
print(f"\nSimulated output voltage: {v_out_ss:.2f} V")
print(f"Expected: {Vout_calc:.2f} V")

## 6. Coupled Inductors

Coupled inductors are used in:
- Multi-phase converters (ripple cancellation)
- SEPIC/Cuk converters
- EMI filters

The coupling coefficient k determines behavior:
$$M = k\sqrt{L_1 L_2}$$

Where M is mutual inductance.

In [ ]:
# Coupled inductor demonstration
# When current changes in L1, voltage is induced in L2

netlist = '''
{
  "name": "Coupled Inductors",
  "components": [
    {"type": "V", "name": "Vpulse", "nodes": ["in", "0"],
     "waveform": {"type": "pulse", "v1": 0, "v2": 5, "td": 0.1e-3,
                  "tr": 10e-6, "tf": 10e-6, "pw": 0.4e-3, "period": 1e-3}},
    {"type": "R", "name": "R1", "nodes": ["in", "p1"], "value": 10},
    {"type": "T", "name": "T1", "nodes": ["p1", "0", "s1", "0"],
     "params": {"turns_ratio": 1.0, "lm": 10e-3, "ll1": 100e-6, "ll2": 100e-6}},
    {"type": "R", "name": "R2", "nodes": ["s1", "0"], "value": 1000}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 2e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Input voltage
axes[0].plot(time_ms, data['signals']['V(in)'], 'b-', linewidth=1.5)
axes[0].set_ylabel('V(in) (V)')
axes[0].set_title('Coupled Inductors - Mutual Inductance Demo')
axes[0].grid(True)

# Primary current (through R1)
v_r1 = np.array(data['signals']['V(in)']) - np.array(data['signals']['V(p1)'])
i_pri = v_r1 / 10
axes[1].plot(time_ms, i_pri, 'g-', linewidth=1.5)
axes[1].set_ylabel('I(primary) (A)')
axes[1].grid(True)

# Induced secondary voltage
axes[2].plot(time_ms, data['signals']['V(s1)'], 'r-', linewidth=1.5)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('V(secondary) (V)')
axes[2].set_title('Induced voltage from changing primary current')
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("Notice: Voltage is induced only during current transitions (di/dt ≠ 0)")

## 7. Core Saturation (Advanced)

Magnetic cores saturate when:
$$B = \frac{V \cdot t}{N \cdot A_e} > B_{sat}$$

Where:
- B = Flux density
- V = Applied voltage
- t = Time
- N = Turns
- Ae = Core cross-section area

Pulsim supports saturation modeling through the magnetic core models.

In [ ]:
# B-H curve illustration
# Simplified Jiles-Atherton model visualization

def bh_curve(H, Ms=1.6, a=500, k=400, c=0.1, alpha=1e-5):
    """Simplified anhysteretic B-H curve."""
    # Langevin function approximation
    He = H + alpha * Ms * np.tanh(H / a)
    M = Ms * (np.tanh(He / a))
    B = 4 * np.pi * 1e-7 * (H + M * 1e6)  # µ0 * (H + M)
    return B

H = np.linspace(-2000, 2000, 1000)
B = bh_curve(H)

plt.figure(figsize=(10, 6))
plt.plot(H, B, 'b-', linewidth=2)
plt.axhline(y=0.4, color='r', linestyle='--', label='B_sat ≈ 0.4T (ferrite)')
plt.axhline(y=-0.4, color='r', linestyle='--')
plt.xlabel('H (A/m)')
plt.ylabel('B (T)')
plt.title('B-H Curve Showing Saturation')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("Saturation effects:")
print("  - Inductance drops sharply")
print("  - Current increases rapidly")
print("  - Can damage switching devices")
print("  - Must be avoided in normal operation")

In [ ]:
# Calculate volt-seconds to avoid saturation
def calc_max_volt_seconds(Bsat, N, Ae):
    """Calculate maximum volt-seconds before saturation."""
    return 2 * Bsat * N * Ae  # Factor of 2 for bipolar swing

# Example: EE25 core
Ae = 52e-6      # 52 mm² effective area
N = 20          # 20 turns
Bsat = 0.35     # Tesla (ferrite)

Vs_max = calc_max_volt_seconds(Bsat, N, Ae)

print("Transformer Core Design:")
print(f"  Core: EE25 ferrite")
print(f"  Ae = {Ae*1e6:.0f} mm²")
print(f"  N = {N} turns")
print(f"  Bsat = {Bsat} T")
print(f"\n  Max volt-seconds: {Vs_max*1e6:.1f} µV·s")

# For given voltage and frequency
V = 48  # Volts
D_max = 0.5  # Max duty cycle
f_sw = 100e3  # Switching frequency

Vs_applied = V * D_max / f_sw
print(f"\n  At {V}V, {f_sw/1e3:.0f}kHz, D={D_max}:")
print(f"  Applied volt-seconds: {Vs_applied*1e6:.1f} µV·s")

if Vs_applied < Vs_max:
    print(f"  ✓ Design OK (margin: {(1 - Vs_applied/Vs_max)*100:.0f}%)")
else:
    print(f"  ✗ SATURATION RISK!")

## Summary

### Transformer Parameters in Pulsim

```python
TransformerParams:
    turns_ratio  # N1:N2 voltage ratio
    lm           # Magnetizing inductance (0 = ideal)
    ll1          # Primary leakage inductance
    ll2          # Secondary leakage inductance
```

### Key Equations

| Parameter | Formula | Effect |
|-----------|---------|--------|
| Voltage ratio | V1/V2 = N1/N2 | Steps voltage up/down |
| Current ratio | I2/I1 = N1/N2 | Inverse of voltage |
| Magnetizing current | Im = V/(ωLm) | No-load current |
| Leakage energy | E = ½LlI² | Must be managed |
| Coupling | k = M/√(L1L2) | 1 = ideal |

### Design Guidelines

1. **Magnetizing inductance**: Size for acceptable no-load current
2. **Leakage inductance**: Minimize or use for soft-switching
3. **Volt-seconds**: Stay below saturation limit
4. **Core losses**: Consider at high frequency

**Next:** [Switches and Event Detection](10_switches.ipynb)